# Lab 2 - P4 Program Building Blocks

This lab describes the building blocks and the general structure of a P4 program. It maps the program’s components to the Protocol-Independent Switching Architecture (PISA), a programmable pipeline used by modern whitebox switching hardware. The lab also demonstrates how to track an incoming packet as it traverses the pipeline of the switch. Such a capability is very useful to debug and troubleshoot a P4 program. 

<img src="./labs_files/lab1/figs/fabric_topology.png" width="550px"><br>

# Background

## The PISA Architecture

The Protocol Independent Switch Architecture (PISA) is a packet processing model that includes the following elements (see Figure below): 
<ul>
    <li>Programmable parser</li>
    <li>Programmable match-action pipeline</li> 
    <li>Programmable deparser</li>     
</ul>
    
The programmable parser permits the programmer to define the headers (according to custom or standard protocols) and to parse them. The parser can be represented as a state machine. 

The programmable match-action pipeline executes the operations over the packet headers and intermediate results. A single match-action stage has multiple memory blocks (e.g., tables, registers) and Arithmetic Logic Units (ALUs), which allow for simultaneous lookups and actions. Since some action results may be needed for further processing (e.g., data dependencies), stages are arranged sequentially. 

The programmable deparser assembles the packet headers back and serializes them for transmission. A PISA device is protocol independent. The P4 program defines the format of the keys used for lookup operations. Keys can be formed using packet header’s information. 

The control plane populates table entries with keys and action data. Keys are used for matching packet information (e.g., destination IP address) and action data is used for operations (e.g., output port).

<img src="./labs_files/lab2/figs/pisa.PNG" width="650px"><br>

## Programmable parser

The programmable parser permits the programmer to define the headers (according to custom or standard protocols) and to describe how the switch should process those headers. The parser de-encapsulates the headers, converting the original packet into a parsed representation of the packet. The programmer declares the headers that must be recognized and their order in the packet. The parser can be represented as a state machine without cycles (direct acyclic graph), with one initial state (start) and two final states (accept or reject).

## Programmable match-action pipeline

The match-action pipeline implements the processing occurring at a switch. The pipeline consists of multiple identical stages (N stages are shown in Figure 1). Practical implementations may have 10/15 stages on the ingress and egress pipelines. Each stage contains multiple match-action units (4 units per stage in Figure 1). A match-action unit has a match phase and an action phase. During the match phase, a table is used to match a header field of the incoming packet against entries in the table (e.g., destination IP address). Note that there are multiple tables in a stage (4 tables per stage in Figure 1), which permit the switch to perform multiple matches in parallel over different header fields. Once a match occurs, a corresponding action is performed by the ALU. Examples of actions include: modify a header field, forward the packet to an egress port, drop the packet, and others. The sequential arrangement of stages allows for the implementation of serial dependencies. For example, if the result of an operation is needed prior to perform a second operation, then the compiler would place the first operation at an earlier stage than the second operation.  

## Programmable deparser

The deparser assembles back the packet and serializes it for transmission. The programmer specifies the headers to be emitted by the deparser. When assembling the packet, the deparser emits the specified headers followed by the original payload of the packet. 

## The V1Model

The figure below depicts the V1Model architecture components. The V1Model architecture consists of a programmable parser, an ingress match-action pipeline, a traffic manager, an egress match-action pipeline, and a programmable deparser. The traffic manager schedules packets between input ports and output ports and performs packet replication (e.g., replication of a packet for multicasting). The V1Model architecture is implemented on top BMv2’s simple_switch target.

<img src="./labs_files/lab2/figs/v1model.PNG" width="700px"><br>

## P4 program mapping to the V1Model

The P4 program used in this lab is separated into different files. The figure below shows the V1Model and its associated P4 files. These files are as follows:

<ul>
<li>headers.p4: this file contains the packet headers’ and the metadata’s definitions.</li>
<li>parser.p4: this file contains the implementation of the programmable parser.</li>
<li>ingress.p4: this file contains the ingress control block that includes match-action tables.</li>
<li>egress.p4: this file contains the egress control block.</li>
<li>deparser.p4: this file contains the deparser logic that describes how headers are emitted from the switch.</li>
<li>checksum.p4: this file contains the code that verifies and computes checksums.</li>
<li>basic.p4: this file contains the starting point of the program (main) and invokes the other files. This file must be compiled.</li>
    
</ul>

<img src="./labs_files/lab2/figs/v1model_mapping.PNG" width="700px"><br>

# Step 1:  Configure the Environment

Before running this notebook, you will need to configure your environment using the [Configure Environment](../../../configure.ipynb) notebook. Please stop here, open and run that notebook, then return to this notebook.

If you are using the FABRIC JupyterHub many of the environment variables will be automatically configured for you.  You will still need to set your bastion username, upload your bastion private key, and set the path to where you put your bastion private key. Your bastion username and private key should already be in your possession.  

If you are using the FABRIC API outside of the JupyterHub you will need to configure all of the environment variables. Defaults below will be correct in many situations but you will need to confirm your configuration.  If you have questions about this configuration, please contact the FABRIC admins using the [FABRIC User Forum](https://learn.fabric-testbed.net/forums/) 

More information about accessing your experiments through the FABRIC bastion hosts can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).
 

# Step 2: Import the FABlib Library

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

# Step 3: Create the Experiment Slice

The following creates three node with basic compute and networking capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

### Step 3.1: Create a slice
The code below creates a new slice with the name "my_slice"

In [ ]:
slice = fablib.new_slice(name="lab2")

### Step 3.2: Define the sites
The code below requests three sites from FABRIC: GATECH, STAR, and NCSA

<img src="./labs_files/lab1/figs/fabric_sites.png" width="550px"><br>

In [ ]:
site1='MICH'
site2='STAR'
site3='NCSA'

print (f'The selected sites are {site1}, {site2}, {site3}') 

### Step 3.3: Creating the nodes
The code below creates three nodes: server1, switch, and server2. The servers (server1 and server2) use the following
<ul>
    <li> 4 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 20GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

server1 will be created in site1 and server3 will be created in site3

<img src="./labs_files/lab1/figs/creating_nodes.PNG" width="550px"><br>

In [ ]:
server1 = slice.add_node(name="server1", 
                      site=site1, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

server2 = slice.add_node(name="server2", 
                      site=site3, 
                      cores=4, 
                      ram=8, 
                      disk=20, 
                      image='default_ubuntu_20')

### Step 3.4: Adding the interfaces to the servers
The code below adds a Network Interface Card (NIC) to each server.

<img src="./labs_files/lab1/figs/adding_nics.PNG" width="550px"><br>

In [ ]:
server1_iface = server1.add_component(model='NIC_Basic').get_interfaces()[0]
server2_iface = server2.add_component(model='NIC_Basic').get_interfaces()[0]

### Step 3.5: Creating a node for the P4 switch
The code below creates a node that will run the P4 switch. The node use the following
<ul>
    <li> 16 CPU cores</li>
    <li> 8GB RAM </li>
    <li> 40GB disc size </li>
    <li> Image: Ubuntu 20.04
</ul>

The node will be created in site2

<img src="./labs_files/lab1/figs/adding_switch.PNG" width="550px"><br>

In [ ]:
# Add a node
switch = slice.add_node(name="switch", 
                      site=site2, 
                      cores=32, 
                      ram=16, 
                      disk=40, 
                      image='default_ubuntu_20')

### Step 3.6: Adding two interfaces to the switch
The code below adds two Network Interface Cards (NICs) to the switch.

<img src="./labs_files/lab1/figs/adding_switch_ports.PNG" width="550px"><br>

In [ ]:
switch_iface1 = switch.add_component(model='NIC_Basic', name='net1_nic').get_interfaces()[0]
switch_iface2 = switch.add_component(model='NIC_Basic', name='net2_nic').get_interfaces()[0]

### Step 3.7: Connecting site1 and site2
Create a site-to-site network between site1 and site2 connecting server1 and the P4 switch

<img src="./labs_files/lab1/figs/connecting_nodes_server1_switch.PNG" width="550px"><br>

In [ ]:
net1 = slice.add_l2network(name='net1', interfaces=[server1_iface, switch_iface1])

### Step 3.8: Connecting site2 and site3
Create a site-to-site network between site2 and site3 connecting the P4 switch and server2

<img src="./labs_files/lab1/figs/connecting_nodes_server2_switch.PNG" width="550px"><br>

In [ ]:
net2 = slice.add_l2network(name='net2', interfaces=[switch_iface2, server2_iface])

### Step 3.9: Submitting the slice
The code below submits the slice. 
By default, the submit function will block until the node is ready and will display the progress of your slice being built.

In [ ]:
#Submit Slice Request
slice.submit();

# Step 4: Installing the required packages
In this step, we will install the required packages to run the labs. Specifically, we will install the BMv2 software switch and its control plane, the P4 compiler (p4c), and net-tools.


## Step 4.1 Installing BMv2
The BMv2 software switch will be installed on the switch node. We will upload the script [scripts/install_bmv2.sh](./scripts/install_bmv2.sh) to the switch and execute it

In [ ]:
switch = slice.get_node(name="switch")     
switch.upload_file('scripts/install_bmv2.sh', 'install_bmv2.sh')
stdout, stderr = switch.execute(f'chmod +x install_bmv2.sh &&  ./install_bmv2.sh',quiet=True)

## Step 4.2 Installing net-tools
The net-tools package will be installed on the switch, server1 and server2 nodes. This package will allow us to use the ifconfig and the arp commands 

In [ ]:
server1 = slice.get_node(name="server1")
server2 = slice.get_node(name="server2")
stdout, stderr = server1.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get install -y net-tools', quiet=True)
stdout, stderr = switch.execute(f'sudo apt-get install -y net-tools', quiet=True)

## Step 4.3 Installing scapy
Installing scapy to be able to craft and send raw packets on the servers


In [ ]:
stdout, stderr = server1.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)
stdout, stderr = server2.execute(f'sudo apt-get update && sudo apt-get install -y python3-scapy', quiet=True)

# Step 5: Assigning IP and MAC addresses
In this step, we will assign IPv4 addresses to the interfaces of the servers and the switch. We will also hardcode the MAC addresses. 

## Step 5.1: Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/lab1/figs/interfaces.PNG" width="550px"><br>

In [ ]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()
print(f'server1_iface: {server1_iface_name}')

node2_iface = server2.get_interface(network_name='net2') 
server2_iface_name = node2_iface.get_device_name()
print(f'server2_iface: {server2_iface_name}')

switch_iface1 = switch.get_interface(network_name='net1') 
switch_iface1_name = switch_iface1.get_device_name()
print(f'switch_iface1: {switch_iface1_name}')

switch_iface2 = switch.get_interface(network_name='net2') 
switch_iface2_name = switch_iface2.get_device_name()
print(f'switch_iface2: {switch_iface2_name}')

## Step 5.2: Turning all interfaces up
In this step, we will use the ip link command to turn the interfaces up

<img src="./labs_files/lab1/figs/interfaces_up.PNG" width="550px"><br>

In [ ]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface1_name} up', quiet=True)
stdout, stderr = switch.execute(f'sudo ip link set dev {switch_iface2_name} up', quiet=True)

## Step 5.3: Hardcode MAC addresses
For simplicity, we will use the following MAC addresses for the interfaces:
<ul>
    <li> server1_iface_MAC = '00:00:00:00:00:01' (shown as 00:01 in the figure below) </li>
    <li>switch_iface1_MAC = '00:00:00:00:00:02' (shown as 00:02 in the figure below)</li>
    <li>switch_iface2_MAC = '00:00:00:00:00:03' (shown as 00:03 in the figure below)</li>
    <li>server2_iface_MAC = '00:00:00:00:00:04' (shown as 00:04 in the figure below)</li>
</ul>

<img src="./labs_files/lab1/figs/mac_addresses.PNG" width="550px"><br>

In [ ]:
server1_iface_MAC = '00:00:00:00:00:01'
switch_iface1_MAC = '00:00:00:00:00:02'
switch_iface2_MAC = '00:00:00:00:00:03'
server2_iface_MAC = '00:00:00:00:00:04'

## Step 5.4 Configuring the IP and MAC addresses on server1_iface and switch_iface1

We will use the network 192.168.1.0/24 between Site1 and Site2. We will assign the IP address 192.168.1.10 to server1's interface and 192.168.1.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_1.PNG" width="550px"><br>

In [ ]:
server1 = slice.get_node(name="server1")     

server1_switch_subnet = "192.168.1.0/24"
server1_ip = '192.168.1.10/24'
switch_ip1 = '192.168.1.1/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} {switch_ip1}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface1_name} hw ether {switch_iface1_MAC}')

## Step 5.5: Configuring the IP and MAC addresses on switch_iface2 and server2_iface

We will use the network 192.168.2.0/24 between Site2 and Site3. We will assign the IP address 192.168.2.10 to server2's interface and 192.168.2.1 to its neighboring interface on the switch.

<img src="./labs_files/lab1/figs/IPs_2.PNG" width="550px"><br>

In [ ]:
server2 = slice.get_node(name="server2")     

server2_switch_subnet = "192.168.2.0/24"
server2_ip = '192.168.2.10/24'
switch_ip2 = '192.168.2.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} {server2_ip}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} {switch_ip2}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface_name} hw ether {server2_iface_MAC}')
stdout, stderr = switch.execute(f'sudo ifconfig {switch_iface2_name} hw ether {switch_iface2_MAC}')

# Step 6: Configure forwarding and routing

## Step 6.1: Enable forwarding on the switch

The command "sudo sysctl -w net.ipv4.ip_forward=1" is used to enable IP forwarding on a Linux system.

IP forwarding is a feature that allows a system to act as a router by forwarding network packets from one network interface to another. By default, IP forwarding is usually disabled on Linux systems for security reasons. 

The command will be executed on the switch device.

In [ ]:
command = 'sudo sysctl -w net.ipv4.ip_forward=1' 
stdout, stderr = switch.execute(command, quiet=True)

## Step 6.2: Delete routing entries for the routes to force traffic to go through the BMv2 switch

In this step, we are deleting the routes on the switch's routing table in Linux. By deleting the routes, the packets will go through the BMv2 switch instead of being forwarded by the kernel

In [ ]:
stdout, stderr = switch.execute(f'sudo ip route del {server1_switch_subnet}', quiet=True)
stdout, stderr = switch.execute(f'sudo ip route del {server2_switch_subnet}', quiet=True)

## Step 6.3: Configure routing

In this step, we will configure static routes on server1 and server2. 
<ul>
    <li> For server1, we will add a route to reach the network 192.168.2.0/24 via 192.168.1.1 </li>
    <li> For server2, we will add a route to reach the network 192.168.1.0/24 via 192.168.2.1 </li>
</ul>

<img src="./labs_files/lab1/figs/routing.PNG" width="550px"><br>

In [ ]:
gw1 = switch_ip1.split('/')[0]
gw2 = switch_ip2.split('/')[0]
stdout, stderr = server1.execute(f'sudo ip route add {server2_switch_subnet} via {gw1}')
stdout, stderr = server2.execute(f'sudo ip route add {server1_switch_subnet} via {gw2}')

## Step 6.4: Configure ARP

In this step, we will configure static ARP entries on server1 and server2. The reason we are doing this is because the switch does not process ARP packets unless programmed to. To make sure that ARP packets are not sent towards the switch, we will hardcode the MACs on the servers.

For each server, we will add an ARP entry to its switch's neighboring interface.

In [ ]:
stdout, stderr = server1.execute(f'sudo arp -s {gw1} {switch_iface1_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {gw2} {switch_iface2_MAC}')

# Step 7:  Navigating through the components of a basic P4 program

This section shows the steps required to compile the P4 program. It illustrates the editor that will be used to modify the P4 program, and the P4 compiler that will produce a data plane program for the software switch. 
 

# Step 7.1: Describing the components of the P4 program

Click on [basic.p4](./labs_files/lab2/src/basic.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/basic.png" width="550px"><br>

The basic.p4 file includes the starting point of the P4 program and other files that are specific to the language (core.p4) and to the architecture (v1model.p4). To make the P4 program easier to read and understand, we separated the whole program into different files. To use those files, the main file (basic.p4) must include them first. For example, to use the parser, we need to include the parser.p4 file (#include “parser.p4”).

We will navigate through the files in sequence as they appear in the architecture.

<hr>

Click on [headers.p4](./labs_files/lab2/src/headers.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/headers.png" width="550px"><br>

The headers.p4 above shows the headers that will be used in our pipeline. We can see that the ethernet and the IPv4 headers are defined. We can also see how they are grouped into a structure (struct headers). The headers name will be used throughout the program when referring to the headers. Furthermore, the file shows how we can use typedef to provide an alternative name to a type.

<hr>

Click on [parser.p4](./labs_files/lab2/src/parser.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/parser.png" width="550px"><br>

The figure above shows the content of the parser.p4 file. We can see that the parser is already written with the name MyParser. This name will be used when defining the pipeline sequence. 

<hr>

Click on [ingress.p4](./labs_files/lab2/src/ingress.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/ingress.png" width="550px"><br>

The figure above shows the content of the ingress.p4 file. We can see that the ingress is already written with the name MyIngress. This name will be used when defining the pipeline sequence.

<hr>

Click on [egress.p4](./labs_files/lab2/src/egress.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/egress.png" width="550px"><br>

The figure above shows the content of the egress.p4 file. We can see that the egress is already written with the name MyEgress. This name will be used when defining the pipeline sequence. 

<hr>

Click on [checksum.p4](./labs_files/lab2/src/checksum.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/checksum.png" width="550px"><br>

The figure above shows the content of the checksum.p4 file. We can see that the checksum is already written with two control blocks: MyVerifyChecksum and MyComputeChecksum. These names will be used when defining the pipeline sequence. Note that MyVerifyChecksum is empty since no checksum verification is performed in this lab.

<hr>

Click on [deparser.p4](./labs_files/lab2/src/deparser.p4) to open the file in the editor.

<img src="./labs_files/lab2/figs/deparser.png" width="550px"><br>

The figure above shows the content of the deparser.p4 file. We can see that the deparser is already written with two instructions that reassemble the packet.

# Step 7.2: Programming the pipeline sequence 

Now it is time to write the pipeline sequence in the basic.p4 program. 

Click on [basic.p4](./labs_files/lab2/src/basic.p4) to open the file in the editor.

Write the following block of code at the end of the file:

    V1Switch (
        MyParser(),
        MyVerifyChecksum(),
        MyIngress(),
        MyEgress(),
        MyComputeChecksum(),
        MyDeparser()
    ) main;
  
<img src="./labs_files/lab2/figs/basic_filled.png" width="500px"><br>

We can see here that we are defining the pipeline sequence according to the V1Model architecture. First, we start by the parser, then we verify the checksum. Afterwards, we specify the ingress block and the egress block, and we recompute the checksum. Finally, we specify the deparser.

**Save the changes by pressing Ctrl+s.**

# Step 8: Uploading and Running the P4 program to the switch

In this step, we upload the P4 program to the switch, compile it, and start the switch daemon. 

## Step 8.1: Uploading the P4 program

The P4 program [basic.p4](labs_files/lab2/src/basic.p4) is located under lab_files/lab2/src.

We will be uploading the whole directory since it includes other P4 files. 

In [ ]:
switch = slice.get_node(name='switch')        
switch.upload_directory('labs_files/lab2/src', '/home/ubuntu/lab2')

## Step 8.2: Compiling the P4 program

In this step, we will use the p4c compiler to compile the program.

Launch a new terminal by clicking on "File" -> "New" -> "Terminal".

<img src="./labs_files/lab2/figs/terminal.gif" width="600px"><br>

Copy the output of the command below and paste into the terminal to enter to the switch.

In [ ]:
switch.get_ssh_command()

Run the following commands in the terminal:
    
    sudo su
    p4c lab2/src/basic.p4
    simple_switch -i 0@ens8 -i 1@ens7 basic.json --log-console
    
<img src="./labs_files/lab2/figs/daemon.png" width="750px"><br>

## Step 8.3: Populating table from the control plane

In this step we will populate the forwarding table by executing a script. We will learn how to populate the tables manually in another lab.

In [ ]:
switch.upload_file('labs_files/lab2/rules.sh', 'rules.sh')
stdout, stderr = switch.execute('chmod +x rules.sh && ./rules.sh')

## Step 8.4: Sending a packet from server1 to the switch

In this step, we will send a packet from server1 to server2. The packet will be processed in the switch. 

In [ ]:
server1.upload_file('labs_files/lab2/src/send.py', 'send.py')
#server1.execute(f'sudo python3 send.py {server1_iface} 192.168.1.10 192.168.2.10 HelloWorld')
stdout, stderr = server1.execute(f'sudo python3 send.py ens7 192.168.1.10 192.168.2.10 HelloWorld')

## Step 8.5: Inspect the logs on the switch

Go back to the switch terminal and inspect the logs.

<img src="./labs_files/lab2/figs/switch_output.png" width="750px"><br>

The figure above shows the processing logic as the packet enters the switch. The packet
arrives on port 0, then the parser starts extracting the headers. After the parsing is done, the packet is processed in the ingress and in the egress pipelines. Then, the checksum update is executed and the deparser reassembles and emits the packet
using port 1 (port_out: 1).

## Step 9: Delete the Slice

This concludes Lab 2. Please delete your slice when you are done with your experiment.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="lab2")
slice.delete()